In [ ]:
%load_ext autoreload
%autoreload 2
import os
from tqdm.notebook import tqdm
from mani_skill.examples.objaverse_handler import SpokDatasetBuilder
N_samples = 20_000
spock_dataset = SpokDatasetBuilder(maximum_objects=N_samples)
print("dataset size", len(spock_dataset))
scored_descr = spock_dataset.spok_root_path / "clip_description.jsonl"

In [ ]:
# this should be moved to the code that saves clip_description.jsonl
import json
scored_descr = spock_dataset.spok_root_path / "clip_description.jsonl"
new_scored_descr = spock_dataset.spok_root_path / "clip_description2.jsonl"

data = []
uuids_seen = set()
with open(scored_descr, "r") as f_obj:
    for line in f_obj:
        entry = json.loads(line)
        if entry["uid"] in uuids_seen:
            print("removing duplicate")
            continue
        data.append(entry)
        uuids_seen.add(entry["uid"])
print(len(uuids_seen))

for entry in data:
    bbox = spock_dataset.get_bbox(entry["uid"])
    entry["bbox"] = bbox.tolist()

with open(new_scored_descr, "w") as f_obj:
    for entry in data:
        f_obj.write(json.dumps(entry) + "\n")
print("done saving.")

In [ ]:
import json
import numpy as np
from tqdm.notebook import tqdm
from heapq import nsmallest

def top_n_above_threshold(scores, t, n):
    return nsmallest(n, ((k, v) for k, v in scores.items() if v > t), key=lambda x: x[1])

grayscale_scores = {}
low_poly_scores = {}
best_descr_scores = {}
best_descr = {}

data = []
with open(scored_descr, "r") as f_obj:
    for line in f_obj:
        data.append(json.loads(line))
        
print("dataset size", len(data))
for annotation in data:
    # Parse each line as a JSON object
    low_poly_scores[annotation["uid"]] = annotation["quality"]["cartoon low-poly model"]
    grayscale_scores[annotation["uid"]] = annotation["quality"]["grayscale image"]
    
    short_descr_values = [v for k,v in annotation["description"].items() if len(k.split()) < 4]
    short_descr_text = [k for k,v in annotation["description"].items() if len(k.split()) < 4]
    
    best_descr_scores[annotation["uid"]] = max(short_descr_values)
    best_descr[annotation["uid"]] = short_descr_text[np.argmax(short_descr_values)]

In [ ]:
#import matplotlib.pyplot as plt
#plt.hist(low_poly_scores.values(), bins=20)
#plt.show()

In [ ]:
from PIL import Image

#threshold_uids = top_n_above_threshold(grayscale_scores, -12, 20)
threshold_uids = top_n_above_threshold(low_poly_scores, -10, 20)
#threshold_uids = top_n_above_threshold(best_descr_scores, -3, 20)

for uid, values in threshold_uids:
    image_files = sorted( list((spock_dataset.spok_models_path / uid / "blender_renders" ).iterdir()))
    image = Image.open(image_files[0])
    print(best_descr[uid])
    display(image)
    break

In [ ]:
from typing import List, Dict, Literal, Optional

def filter_func(annotation: Dict, t_best_descr=-3, t_low_poly=-10, t_grayscale=-12):
        v_low_poly = annotation["quality"]["cartoon low-poly model"]
        v_grayscale = annotation["quality"]["grayscale image"]
        if v_low_poly > t_low_poly:
            return False
        if v_grayscale > t_grayscale:
            return False
        
        short_descr_values = [v for k,v in annotation["description"].items() if len(k.split()) < 4]
        v_best_descr = max(short_descr_values)
        if v_best_descr < t_best_descr:
            return False

        short_descr_text = [k for k,v in annotation["description"].items() if len(k.split()) < 4]
        best_descr = short_descr_text[np.argmax(short_descr_values)]
        blacklist_words = set(("stone", "rock", "figurine", "ring"))
        if set(best_descr.split()).intersection(blacklist_words):
            return False
        return True
    
print("dataset size:", len(data))
data_filtered = [annotation for annotation in data if filter_func(annotation)]
print("dataset size (filtered):", len(data_filtered))

In [ ]:
from collections import Counter
words = []
for annotation in data_filtered:
    words.extend(best_descr[annotation["uid"]].split())
Counter(words).most_common(50)

In [ ]:
import random

n = 2
for annotation in random.sample(data_filtered, min(n, len(data_filtered))):
    uid = annotation["uid"]
    image_files = sorted(list((spock_dataset.spok_models_path / uid / "blender_renders" ).iterdir()))
    image = Image.open(image_files[0])
    print(best_descr[uid])
    display(image)

# uid = "3239828896624cdaae337e1b8b5ca78f"
# image_files = sorted(list((spock_dataset.spok_models_path / uid / "blender_renders" ).iterdir()))
# image = Image.open(image_files[0])
# print(best_descr[uid])
# display(image)

In [ ]:
import random
Counter([best_descr[annotation["uid"]] for annotation in data_filtered]).most_common(30)